In [ ]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np

from attila.data.parse import parse_data, get_data
from attila.experiments.do import do_batch_experiments, do_experiment
from attila.util.config import get_env
from attila.util.io import load_json, stuff2pickle
from attila.util.plots import extract_preds

In [ ]:
_here = Path('.').resolve()

In [ ]:
config, data_path, out_path, _ = get_env(_here)
out_path.mkdir(parents=True, exist_ok=True)  # rm and mkdir if existing

images_path = data_path / config.get('data', 'images')
masks_path = data_path / config.get('data', 'masks')

In [ ]:
raw = get_data(images_path, masks_path)
X, y = parse_data(
    raw,
    (config.getint('image', 'width'), config.getint('image', 'height'))
)

In [ ]:
config.set('experiments', 'test size', '0.95')  # or any other big amount (< 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=config.getfloat('experiments', 'test size'),
    random_state=42  # reproducible results
)
print('train/val data: X ~ {}, y ~ {}'.format(X_train.shape, y_train.shape))
print('test data: X ~ {}, y ~ {}'.format(X_test.shape, y_test.shape))

num_plots = 4
plot_ids = np.random.randint(len(X_test), size=num_plots)

In [ ]:
experiment = {
  "use_skip_conn": True,
  "padding": "same",
  "use_se_block": False,
  "name": "with_same"
}

config.set('experiments', 'val size', '0.5')  # not very necessary to validate
config.set('training', 'batch size', '2')  # very mini-batch size
config.set('training', 'epochs', '20')  # or any other small amount

In [ ]:
config.set('data', 'aug', 'False')

summary = do_experiment(experiment, (X_train, X_test, y_train, y_test), 0, config, plot_ids)

out_folder = out_path / 'trials' / 'to-aug-or-not' / 'no-aug'
out_folder.mkdir(parents=True, exist_ok=True)
out_f = out_folder / config.get('experiments', 'output file')
stuff2pickle(summary, out_f)

In [ ]:
config.set('data', 'aug', 'True')

summary = do_experiment(experiment, (X_train, X_test, y_train, y_test), 0, config, plot_ids)

out_folder = out_path / 'trials' / 'to-aug-or-not' / 'aug'
out_folder.mkdir(parents=True, exist_ok=True)
out_f = out_folder / config.get('experiments', 'output file')
stuff2pickle(summary, out_f)

In [ ]:
for folder in dirs(out_path / 'trials' / 'to-aug-or-not'):
  summary = get_summary(folder, config)
  plot_history(
    summary['history'],
    out_folder=folder
  )
  print('history img saved in {}'.format(model_folder))